In [1]:
import math
import random

import numpy as np

In [2]:
class RLAgent:
    interpreter = None
    policy = None

    _trace = []
    
    def __init__(self, interpreter, policy):
        self.interpreter = interpreter
        self.policy = policy
        self._trace = []
    
    def step(self, h):
        state = self.interpreter.state()
        actions = state.actions()
        if len(actions) != 0:
            random.shuffle(actions)
            action = self.policy.next_action(h, state, actions)
            if action != None:
                self.interpreter.do(action)
                next_state = self.interpreter.state()
                self.policy.update(h, state, action, next_state)
                self._trace.append((state, action))
    
    def next_episode(self):
        self.policy.update_episode(self._trace)
        self._trace = []

In [3]:
class ExplorationPolicy:
    horizon = 0
    state_space = 0
    episode = 0
    actions = 0
    
    prob = 0
    c = 1
    eta = 0.0

    epsilon = 0.0
    rand = random.Random()

    state = None
    
    
    def __init__(self, horizon, state_space, episode, actions, prob, c, epsilon=0.0):
        self.horizon = horizon
        self.state_space = state_space
        self.episode = episode
        self.actions = actions
        self.prob = prob
        self.c = c
        self.epsilon = epsilon
        
        self.eta = math.log10(self.horizon * self.state_space * self.episode * self.actions / float(self.prob))
        self.state = ExplorationState(self.horizon, self.c, self.eta)
    
    def next_action(self, step, state, actions):
        if self.rand.random() < self.epsilon:
            return random.choice(actions)
        return self.state.next_action(step, state, actions)
    
    def update(self, step, state, action, next_state):
        self.state.update(step, state, action, next_state)
    
    def update_episode(self, trace):
        return None
    
class ExplorationState:
    horizon = 0
    c = 0
    eta = 0
    
    qa_map = {}
    visits = {}
    state_values = {}

    _default_qvalue = 0
    
    def __init__(self, horizon, c, eta):
        self.horizon = horizon
        self.c = c
        self.eta = eta
        self.qa_map = {}
        self.visits = {}
        self.state_values = {}

        self._default_qvalue = self.horizon 
        # + (2*self.c * math.sqrt(math.pow(self.horizon, 3)*(self.eta))) + 100
    
    
    def next_action(self, step, state, actions):
        
        state_key = state.to_string()
        if step not in self.qa_map:
            self.qa_map[step] = {}
        
        if state_key not in self.qa_map[step]:
            self.qa_map[step][state_key] = {}
            
        max_val = 0
        max_action = ""
        
        for a in actions:
            if a not in self.qa_map[step][state_key]:
                self.qa_map[step][state_key][a] = self._default_qvalue
        
        for a in actions:
            val = self.qa_map[step][state_key][a]
            if val > max_val:
                max_val = val
                max_action = a
        
        return max_action
    
    def _init(self, step, state, action = None):
        state_key = state.to_string()
        if step not in self.qa_map:
            self.qa_map[step] = {}
            self.qa_map[step][state_key] = {}
            if action is not None:
                self.qa_map[step][state_key][action] = self._default_qvalue

        if step not in self.visits:
            self.visits[step] = {}
            self.visits[step][state_key] = {}
            if action is not None:
                self.visits[step][state_key][action] = 0

        if step not in self.state_values:
            self.state_values[step] = {}
        
        if state_key not in self.qa_map[step]:
            self.qa_map[step][state_key] = {}
            if action is not None:
                self.qa_map[step][state_key][action] = self._default_qvalue

        if state_key not in self.visits[step]:
            self.visits[step][state_key] = {}
            if action is not None:
                self.visits[step][state_key][action] = 0

        if action is not None:
            if action not in self.qa_map[step][state_key]:
                self.qa_map[step][state_key][action] = self._default_qvalue
            if action not in self.visits[step][state_key]:
                self.visits[step][state_key][action] = 0
    
    def update(self, step, state, action, next_state):
        self._init(step, state, action)
        self._init(step+1, next_state)

        state_key = state.to_string()
        next_state_key = next_state.to_string()

        t = self.visits[step][state_key][action] + 1
        self.visits[step][state_key][action] = t

        max_val = 0
        for a in self.qa_map[step+1][next_state_key]:
            action_val = self.qa_map[step+1][next_state_key][a]
            if action_val > max_val:
                max_val = action_val
        if len(self.qa_map[step+1][next_state_key]) == 0:
            max_val = self.horizon

        next_state_val = max_val
        if next_state_val > self.horizon:
            next_state_val = self.horizon
        
        if step+1 == self.horizon:
            next_state_val = 0
        
        self.state_values[step+1][next_state_key] = next_state_val
        cur_qa_val = self.qa_map[step][state_key][action]
        alpha_t = float(self.horizon + 1)/float(self.horizon + t)
        b_t = self.c * math.sqrt(math.pow(self.horizon, 3)*(self.eta) / t)

        new_qa_val = ((1 - alpha_t) * cur_qa_val) + (alpha_t * (next_state_val + 2 * b_t))
        self.qa_map[step][state_key][action] = new_qa_val
                
    

In [4]:
class RandomPolicy:
    def next_action(self, step, state, actions):        
        i = random.randrange(len(actions))
        return actions[i]
    
    def update(self, step, state, action, next_state):
        return None
    
    def update_episode(self, trace):
        return None


In [5]:
class NegativeRewardPolicy:
    alpha = 0
    gamma = 0
    qa_map = {}

    def __init__(self, alpha, gamma):
        self.alpha = alpha
        self.gamma = gamma

        self.qa_map = {}

    def next_action(self, step, state, actions):
        state_key = state.to_string()
        if state_key not in self.qa_map:
            self.qa_map[state_key] = {}

        sum = 0
        weights = []
        for a in actions:
            if a not in self.qa_map[state_key]:
                self.qa_map[state_key][a] = 0
            val = math.exp(self.qa_map[state_key][a])
            sum += val
            weights.append(val)
        
        for i in range(len(actions)):
            weights[i] = weights[i] / float(sum)
        
        return np.random.choice(actions, p=weights)
    
    def update(self, step, state, action, next_state):
        return None
    
    def update_episode(self, trace):
        for (state, action) in trace:
            state_key = state.to_string()
            if state_key not in self.qa_map:
                continue

            if action not in self.qa_map[state_key]:
                self.qa_map[state_key][action] = 0

            cur_val = self.qa_map[state_key][action]

            max = 0
            for a in self.qa_map[state_key]:
                if self.qa_map[state_key][a] > max:
                    max = self.qa_map[state_key][a]

            next_val = (1 - float(self.alpha)) * cur_val + float(self.alpha) * (-1 + float(self.gamma) * max)
            self.qa_map[state_key][action] = next_val

        

In [6]:
class GridState:
    size = 0
    pos = (0,0)
    blocked_row = 0
    
    def __init__(self, pos, size, blocked_row):
        self.pos = pos
        self.size = size
        self.blocked_row = blocked_row
    
    def actions(self):
        (i,j) = self.pos
        actions = []
        if i > 0:
            actions.append("up")
        if i < (self.size-1):
            actions.append("down")
        if j > 0 and j != self.blocked_row:
            actions.append("left")
        if j < (self.size-1):
            actions.append("right")
        return actions
    
    def to_string(self):
        (i,j) = self.pos
        return f'({i},{j})'
    

        

In [7]:
class GridInterpreter:
    grid = []
    cur_pos = (0,0)
    size = 0
    
    def __init__(self, size):
        self.grid = [[0 for i in range(size)] for i in range(size)]
        self.cur_pos = (0,0)
        self.grid[0][0] = 1
        self.size = size
    
    def state(self):
        return GridState(self.cur_pos, self.size)
    
    def do(self, action):
        (i, j) = self.cur_pos
        if action == "left":
            self.cur_pos = (i, j-1)
        elif action == "right":
            self.cur_pos = (i, j+1)
        elif action == "up":
            self.cur_pos = (i-1, j)
        elif action == "down":
            self.cur_pos = (i+1, j)
        (i,j) = self.cur_pos
        self.grid[i][j] = 1

    def reset(self):
        self.cur_pos = (0,0)

    def covered_positions(self):
        count = 0
        for i in range(self.size):
            for j in range(self.size):
                if self.grid[i][j] == 1:
                    count = count +1
        return count

In [ ]:
class DistortedGridInterpreter:
    grid = []
    cur_pos = (0,0)
    size = 0
    narrow_row = 0
    
    def __init__(self, size):
        self.grid = [[0 for i in range(size)] for i in range(size)]
        self.cur_pos = (0,0)
        self.grid[0][0] = 1
        self.size = size
        self.narrow_row = size/2
    
    def state(self):
        return GridState(self.cur_pos, self.size)
    
    def do(self, action):
        (i, j) = self.cur_pos
        if action == "left":
            self.cur_pos = (i, j-1)
        elif action == "right":
            self.cur_pos = (i, j+1)
        elif action == "up":
            self.cur_pos = (i-1, j)
        elif action == "down":
            self.cur_pos = (i+1, j)
        (i,j) = self.cur_pos
        self.grid[i][j] = 1

    def reset(self):
        self.cur_pos = (0,0)

    def covered_positions(self):
        count = 0
        for i in range(self.size):
            for j in range(self.size):
                if self.grid[i][j] == 1:
                    count = count +1
        return count

In [15]:
def get_policy(params):
    policy = params["policy"]
    if policy == "ucbzero":
        horizon = params["horizon"]
        state_space = params["state_space"]
        actions = params["actions"]
        episodes = params["episodes"]

        prob = params["prob"]
        c = params["c"]
        grid_size = params["grid_size"]
        epsilon = params["epsilon"]
        return ExplorationPolicy(horizon, state_space, episodes, actions, prob, c, epsilon)
    elif policy == "softmax":
        alpha = params["alpha"]
        gamma = params["gamma"]
        return NegativeRewardPolicy(alpha, gamma)
    elif policy == "random":
        return RandomPolicy()
    else:
        return None


def run_rl(params):
    policy = get_policy(params)
    horizon = params["horizon"]
    episodes = params["episodes"]
    grid_size = params["grid_size"]
    
    grid_interpreter = GridInterpreter(grid_size)

    rl_agent = RLAgent(grid_interpreter,policy)

    for i in range (episodes):
        for j in range(horizon):
            rl_agent.step(j)
        rl_agent.next_episode()
        
        grid_interpreter.reset()
    
    return grid_interpreter.covered_positions()


In [16]:
def average_coverage(params, runs):
    sum = 0
    for i in range(runs):
        sum += run_rl(params)

    avg = sum/runs
    print("Average coverage: ",avg)
    return avg

def compare(policy1, policy2, runs):
    coverage1 = average_coverage(policy1, runs)
    coverage2 = average_coverage(policy2, runs)

    print("Average coverage 1: ", coverage1)
    print("Average coverage 2: ", coverage2)
    
# average_coverage({"policy": "ucbzero","horizon": 100, "state_space": 400, "actions": 4, "episodes": 1000, "prob": 0.2, "c": 0.001, "grid_size": 20, "epsilon": 0.1, "alpha": 0.3, "gamma": 0.7}, 10)

compare({"policy": "random", "horizon": 100, "episodes": 1000, "grid_size": 20}, {"policy": "ucbzero","horizon": 100, "state_space": 400, "actions": 4, "episodes": 1000, "prob": 0.2, "c": 0.001, "grid_size": 20, "epsilon": 0.1, "alpha": 0.3, "gamma": 0.7}, 10)

Average coverage:  369.1
Average coverage:  357.5
Average coverage 1:  369.1
Average coverage 2:  357.5
